In [58]:
import sys, os
sys.path.insert(1,os.path.abspath('../src'))

Import our code under test.

In [59]:
from fpipeline import pipeline, stepfn, conditionfn, variables

## Test Framework
    

In [60]:
from dataclasses import dataclass, field
from IPython.display import display

@dataclass
class Test:
    name: str
    result: any
    def _success(self):
        display({"text/html": f"✅: {self.name}"}, raw=True)
    def _failure(self):
        display(
            {"text/html": f"<span style='color:red'>❌: {self.name}</span>"}, raw=True)
    def equals(self, value: any):
        if value == self.result:
            self._success()
        else:
            self._failure()

In [61]:
Test("Test Test", None).equals(None)

✅: Test Test

## Test Data Setup

In [62]:
@dataclass
class ctx:
    trace: list[list] = field(default_factory=list)

@stepfn
def tstep(data: ctx, *args, **kwargs):
    d = (*args, {**kwargs})
    data.trace.append(d)
    return data

In [63]:
Test("@stepfn step should take a single argument",
    tstep(7, 8).__code__.co_argcount
).equals(1)

✅: @stepfn step should take a single argument

In [64]:
Test("@stepfn capture args",
    tstep(7, 5)(ctx()).trace
).equals([(7, 5, {})])

✅: @stepfn capture args

In [65]:
Test("Simple pipeline",
    pipeline(tstep(7), tstep(78))(ctx()).trace
 ).equals([(7, {}), (78, {})])

✅: Simple pipeline

In [57]:
def f1():
    with variables(ctx()) as vars:
        return vars.variables
Test("With no variables",
    f1()
).equals({})

✅: With no variables